# Synthesizer
### Arnaud Gaudard, Julien Sahli, Corentin Junod

This notebook presents our projet functionalities.
The synthesizer comes also with a GUI showing the output frequencies in real time, it is not displayed in this notebook. To use the program with all its functionalities, please run it as mentioned in the README. 

All the requirements for the project to run are listed in `requirements.txt`. To install them run `pip install -r requirements.txt`

When a cell is run, the synth activates and start playing. To stop it, press "Enter".


In [ ]:
import os
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
import pygame.midi
import mido
import current_script
import parameters
import utils

import sounddevice as sd
import numpy as np

from multiprocessing import Queue, Process
from Synthesizer import Synthesizer
from FilePlayer import FilePlayer

"""
PARAMETERS
"""
parameters.INPUT_MIDI_DEVICE = 1
parameters.OUTPUT_DEVICE = 'pulse' # sd.default.device is the default value
# 0 = Equal Temperament
# 1 = Pythagorean Tuning
# 2 = Just Intonation
parameters.TEMPERAMENT = 0

"""
Basic function that runs the synth.
It is similar to cli_main.py
"""
def start_synth(MidiToFreq=current_script.MidiToFreq, FreqToAudio=current_script.FreqToAudio):
    pygame.midi.quit()
    pygame.midi.init()
    midi_input = pygame.midi.Input(parameters.INPUT_MIDI_DEVICE)
    
    queue = Queue()
        
    current_script.MidiToFreq = MidiToFreq
    current_script.FreqToAudio = FreqToAudio
    current_script.miditofreq = current_script.MidiToFreq()
    current_script.freqtoaudio = current_script.FreqToAudio()

    synth = Synthesizer(queue, midi_input)
    
    # This file contains a medley of various Mario musics, absolutely perfect for tests
    # If you're not part of the Nintendo fanbase, you may use midi_example2.mid, which is more baroque
    file_player = FilePlayer('ressources/midi_example.mid')
    file_player_proc = Process(target=file_player.run)
    
    # In this notebook, we do not run the GUI
    # gui = GUI(queue)
    # proc = Process(target=gui.run).start()

    print("Running...")
    try:
        file_player_proc.start()
        synth.run()
        
    except KeyboardInterrupt:
        # handle cell being closed
        print("Bye !")
    
    midi_input.close()
    file_player_proc.terminate()

    print("Stopped")

utils.init()
utils.show_peripherals()

<hr style="border-color:black">

## Basic principles

Our synthesizer takes MIDI input in real time or from a file, and convert them into sound. 

The steps are shown in the following diagram. 

<img src="ressources/Process.png"/>

Each step in further detailed in this document.

## Basic example

This is a first basic example generating a simple sine oscillator for each note

In [ ]:
import Modules.Oscillators
"""
This class tells the synth how to convert tuples of frequencies and amplitudes into sound.

It must implement a constructor, where all the oscillators and filters must be declared, it 
is called once when the synth starts.

The method "process" is called for each audio frame.

"""
class FreqToAudio:
    def __init__(self):
        # Oscillators
        self.sine = Modules.Oscillators.Sine()

    """
    Converts a list of frequencies and amplitudes into sound.
    Parameters :
        indexes : An array of indexes of size parameter.SAMPLES_PER_FRAME. 
                  Indicates the indexes of the samples produced by the function, starting from
                  0 for the first sample produced and incrementing over time. This is used by
                  filters operating over time.
        freqs_amps : An array of tuples (frequency, amplitude) that contains en entry per 
                     frequency in herz that must be generated, associated with it amplitude (loudness)
                     from 0 to 1
                     
    Return value :
        A tuple of two arrays (of length parameter.SAMPLES_PER_FRAME) containing the generated 
        samples. The first array is the left ear audio, the second is the right ear audio.
    """
    def process(self, indexes, freqs_amps):
        output = np.zeros(parameters.SAMPLES_PER_FRAME)
        
        # There is no filtering of frequencies and amplitudes in this example
        
        # Oscillators
        for freq, amp in freqs_amps:
            # Security cutting frequencies over the Nyquist frequency
            if freq > parameters.NYQUIST_FREQUENCY: continue
            output += self.sine.set(freq, amp=amp).get(indexes, output)
            
        # There is no filtering of audio signal in this example
        
        # This example is mono
        return output, output 
    
start_synth(FreqToAudio=FreqToAudio)

<hr style="border-color:black">

## Modules

Modules are basic block to construct sounds.
There are 4 kinds of modules :

### 1. Frequencies filtering modules

Those modules take as input the tuples `(frequency, amplitude)` generated by the midi handler.
They apply various operations on the frequencies and output modified tuples of `(frequency, amplitude)`

We implemented the following modules in this categorie :
- ADSR envelope : This module provides an ADSR envelope with parametrized attack, decay, sustain and release
- Resonator : This module add overtones with dampening to a given frequency

### 2. Oscillators

Oscillators take as input the tuples `(frequency, amplitude)`, genrated by the midi handler or a frequency filtering module, and output an audio signal.

We implemented the following modules in this categorie :
- Sine wave
- Square wave
- Sawtooth wave
- Triangle wave
- White noise

### 3. Audio Filters

The audio filters take as input an audio signal and apply operations to it. They output the modified audio signal.

We implemented the following modules in this categorie :
- Biquads
    - Lowpass
    - Highpass
    - Bandpass
    - PeakingEQ
    - Notch
    - Highshelf
    - Lowshelf
    
- Reverb
- Clipping

### 4. Math Modules

Math modules are used to output an audio signal that is not periodic, and thus can not be used to create oscillators. They are usefull to describe functions in the ADSR module.

We implemented the following modules in this categorie :
- Linear
- Exponential

## Piano-like sound

This is an example of a very basic piano sound using additive synthesis, to illustrate modules usage.
As this is a simple case of additive synthesis, we took a C4 on a piano and extracted the overtones.
The sound is not so bad around C4, but becomes quickly "metalic" for high notes. To solve this problem we should use more advance techniques taking into account the changes in overtones with relation to the fundamental.

In [ ]:
import Modules.Oscillators
import Modules.filters.Reverb
import Modules.filters.Biquad.LowPass
import Modules.filters.Biquad.HighPass
import Modules.ADSR
import Modules.Linear
import Modules.Exponential

class FreqToAudio:
    def __init__(self):
        # Filters
        self.reverb = Modules.filters.Reverb.Reverb(delay=0.005, dampening=0.25)
        self.lowpass = Modules.filters.Biquad.LowPass.LowPass(4200,1.5)  # 4200 is just above the highest frequency a piano can do
        # self.highpass = Modules.filters.Biquad.HighPass.HighPass(20,1.5)

        # Oscillators
        self.sine = Modules.Oscillators.Sine()

        # ADSR
        attack_time = 0.005
        attack_stop_level = 0.9
        attack_func = Modules.Linear.Linear(start=0, stop=attack_stop_level, duration=attack_time)
        decay_time = 3
        decay_func=Modules.Exponential.Exponential(start=attack_stop_level, stop=0, duration=decay_time)
        release_time = 0.2
        release_func = Modules.Linear.Linear(attack_stop_level, 0, release_time)
        self.adsr = Modules.ADSR.ADSR(
            attack_time=attack_time, attack_func=attack_func,
            decay_time=decay_time,   decay_func=decay_func,
            sustain_func=0,
            release_time=release_time, release_func=release_func,
        )

    def process(self, indexes, freqs_amps) :
        output = np.zeros(parameters.SAMPLES_PER_FRAME)

        # Frequencies filtering - ADSR
        freqs_amps = self.adsr.get(indexes, freqs_amps)

        # Oscillators
        # overtones taken from a C4 on a piano, but slightly modified
        # the tuples are [overtone_number, amplitude]
        overtones = [
            [0.25, 0.2],
            [0.5, 0.2],
            [1, 1],
            [2, 0.9],
            [3, 0.15],
            [4, 0.39],
            [5, 0.39],
            [6, 0.05],
            [7, 0.05],
            [9, 0.05]
        ]
        for freq, amp in freqs_amps:
            if freq > parameters.NYQUIST_FREQUENCY: continue
            for over_mult, over_amp in overtones:
                output += self.sine.set(freq*over_mult, amp=amp*over_amp).get(indexes, output)

        # Audio filtering
        # output = self.reverb.get(indexes, output)
        output = self.lowpass.get(indexes, output)

        return output, output
    
start_synth(FreqToAudio=FreqToAudio)

<hr style="border-color:black">

## Advanced Module Usage

One of the key aspect of our synthesizer is that any module parameter can be a module itself.
This enables complex modules structures. The modules also supports operators overloading, which emans that it is possible to add, multiply, subtract and divide modules.

We achieve this by having a base classe named `Module` from which any module inherits. When a module is instanciated, it first converts all the numerical parameters into `Constant` modules holding the value. This is done through the method `_param_to_modules` and enable the internal process of a module to only work with modules as parameters.

Each module must expose a function `get` that takes as parameter the audio samples to process (or the frequencies to process in the case of a frequency filtering module or an oscillator) and a list of indexes corresponding to the samples. This list of indexes is used to keep track of time inside a module.


## Flute-like sound

This is an example of a sound close to a flute. It uses oscillators as parameters to modules.

In [ ]:
import Modules.Oscillators
import Modules.filters.Reverb
import Modules.ADSR
import Modules.Linear
import Modules.Exponential
import Modules.filters.Resonator
import Modules.filters.Biquad.LowPass

class FreqToAudio:
    def __init__(self):
        # Filters
        self.reverb = Modules.filters.Reverb.Reverb(delay=0.005, dampening=0.06)
        # Modules parameters can be modules
        self.lowpass = Modules.filters.Biquad.LowPass.LowPass(
            10000, 
            Modules.Oscillators.Sine(0.8, 1.2, 2)
        )

        # Oscillators
        self.sine = Modules.Oscillators.Sine()
        # Example of a LFO
        self.lfo = Modules.Oscillators.Sine(4, 0.1)

        # ADSR
        attack_time = 0.1
        attack_stop_level = 0.9
        attack_func = Modules.Exponential.Exponential(start=0, stop=attack_stop_level, duration=attack_time)
        decay_time = 0
        decay_func=attack_stop_level
        sustain_func=attack_stop_level
        release_time = 0.1
        release_func = Modules.Linear.Linear(attack_stop_level, 0, release_time)
        self.adsr = Modules.ADSR.ADSR(
            attack_time=attack_time, attack_func=attack_func,
            decay_time=decay_time,   decay_func=decay_func,
            sustain_func=sustain_func,
            release_time=release_time, release_func=release_func,
        )

    def process(self, indexes, freqs_amps) :
        output = np.zeros(parameters.SAMPLES_PER_FRAME)

        # Frequencies filtering - ADSR
        freqs_amps = self.adsr.get(indexes, freqs_amps)

        # Oscillators
        # LFO can be used in many ways
        overtones = [
            [0.5, 0.3*(1+self.lfo.get(indexes, output))],
            [1, 1+self.lfo.get(indexes, output)],
            [2, 0.2],
            [3, 0.6*(1+self.lfo.get(indexes+44000, output))],
            [4, 0.02*(1+self.lfo.get(indexes, output))],
            [5, 0.02*(1+self.lfo.get(indexes, output))],
            [6, 0.01],
            [7, 0.01],
            [8, 0.01],
            [9, 0.01]
        ]
        for freq, amp in freqs_amps:
            if freq > parameters.NYQUIST_FREQUENCY: continue
            for over_mult, over_amp in overtones:
                # We transpose one octave up, it renders better with the sound of a flute
                new_freq = freq*over_mult*2
                output += self.sine.set(new_freq, amp=amp*over_amp).get(indexes, output)

        # Audio filtering
        output = self.lowpass.get(indexes, output)
        output = self.reverb.get(indexes, output)

        return output, output
    
start_synth(FreqToAudio=FreqToAudio)

<hr style="border-color:black">

## MIDI Processing

Processing the MIDI input is the taks of the synthesizer.
By default the frequency of a key is calculated using $ 2^{\frac{keyNumber - 69}{12}} \cdot 440 $, where 2 is the frequency ratio between two note one octave apart, 69 is the midi number of A4 and 440 its frequency.

We also implemented just intonation and pythagorean tuning. The temperament can be switched by modifying the value of parameters.TEMPERAMENT (see first python cell of this notebook)



## Subtractive Synthesis

Finally, using all the previous filtering methods, our synthesizer is able to create sounds by subtractive synthesis. As an example we tried to reproduce a sound bringing back .

In [ ]:
import Modules.ADSR
import Modules.Linear
import Modules.Exponential
import Modules.Oscillators


import Modules.filters.Biquad.LowPass
import Modules.filters.Biquad.HighPass

import Modules.filters.Reverb
import Modules.filters.Resonator
import Modules.filters.Comb

class FreqToAudio:
    def __init__(self):
        # Oscillators
        self.square = Modules.Oscillators.Square()
        self.white = Modules.Oscillators.WhiteNoise()
        self.comb = Modules.filters.Comb.Comb(1, 20)
        self.lowpass = Modules.filters.Biquad.LowPass.LowPass(10000, 2)
        self.highpass = Modules.filters.Biquad.HighPass.HighPass(100, 2)
        self.lowpass_hard = Modules.filters.Biquad.LowPass.LowPass(6000, 2)
        self.reverb = Modules.filters.Reverb.Reverb(0.01, 0.4)
        self.lfo = Modules.Oscillators.Sine(4, 0.05)

        # ADSR
        attack_time = 0.07
        attack_stop_level = 0.8
        attack_func = Modules.Linear.Linear(
            start=0, stop=attack_stop_level, duration=attack_time
        )
        decay_time = attack_time
        decay_func = Modules.Linear.Linear(
            start=attack_stop_level, stop=0.7, duration=decay_time
        )
        release_time = 0.1
        release_func = Modules.Linear.Linear(attack_stop_level, 0, release_time)
        self.adsr = Modules.ADSR.ADSR(
            attack_time=attack_time,
            attack_func=attack_func,
            decay_time=decay_time,
            decay_func=decay_func,
            sustain_func=0.7,
            release_time=release_time,
            release_func=release_func,
        )

    def process(
        self,
        indexes,
        freqs_amps,
    ):
        output = np.zeros(parameters.SAMPLES_PER_FRAME)

        freqs_amps = self.adsr.get(indexes, freqs_amps)

        # Oscillators
        for freq, amp in freqs_amps:
            # Security cutting frequencies over the Nyquist frequency
            if freq > parameters.NYQUIST_FREQUENCY: continue
            output += self.white.get(indexes, output) * amp/5
            output += self.square.set(freq,amp+self.lfo.get(indexes, output)).get(indexes, output)

        output = self.comb.get(indexes, output)

        output_reverb = self.lowpass_hard.get(indexes, output)
        output_reverb = self.reverb.get(indexes, output_reverb)
        output = 0.5*output + 0.5*output_reverb

        output = self.highpass.get(indexes, output)
        output = self.lowpass.get(indexes, output)



        return output, output
    
start_synth(FreqToAudio=FreqToAudio)

Running...


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

...

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o